In [1]:
import pandas as pd 

tags = pd.read_csv("Tags.csv")
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
links = pd.read_csv("links.csv")

In [2]:
# Behåll
movies["genres"] = movies["genres"].str.replace("|", " ", regex=False)

In [3]:
# Behåll
tags = tags[tags['tag'].apply(lambda x: isinstance(x, str))]
merged_tags = tags.groupby("movieId")["tag"].apply(lambda x: " ".join(set(x))).reset_index()

In [4]:
# Behåll

filtered_movies = pd.merge(movies, merged_tags, on="movieId", how="inner")
filtered_movies = pd.merge(filtered_movies, links, on="movieId", how="inner")
filtered_movies

,movieId,title,genres,tag,imdbId,tmdbId
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,birthday party Owned very good battery microph...,114709,862.0
1,2,Jumanji (1995),Adventure Children Fantasy,time alternate timeline handcuffs jungle man p...,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy Romance,moldy funny comedy old people that is actually...,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy Drama Romance,chick flick characters girl movie single mothe...,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,Fantasy steve martin Comedy confidence regret ...,113041,11862.0
...,...,...,...,...,...,...
53447,288765,Bird Box Barcelona (2023),Horror Thriller,tw suicide bad science post-apocalyptic surviv...,14400246,805320.0
53448,288779,Don Camillo: Monsignor (1961),Comedy,Don Camillo Series,54814,11580.0
53449,288849,Colaholic (2018),(no genres listed),short film addiction animation,8632066,628759.0
53450,288937,Blue Thermal (2022),Animation Comedy Drama Romance,anime,15304756,865767.0


In [5]:
# Behåll

import re
import unicodedata

def clean_text(text):
    if type(text)==str:
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode() 
        # converts to lowercase
        text = text.lower()
    
        # removes special characters, numbers, and punctuation
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        
    
    return text

In [6]:
# Behåll

text_columns = filtered_movies.columns[filtered_movies.dtypes == 'object']
for col in text_columns:
    filtered_movies[col] = filtered_movies[col].apply(clean_text)

In [10]:
filtered_movies

,movieId,title,genres,tag,imdbId,tmdbId
0,1,toy story 1995,adventure animation children comedy fantasy,catchphrase mockery leggings the boys cult fav...,114709,862.0
1,2,jumanji 1995,adventure children fantasy,lush refrigerator fantasy serial escalation re...,113497,8844.0
2,3,grumpier old men 1995,comedy romance,minnesota nofaganes clv walter matthau sequel ...,113228,15602.0
3,4,waiting to exhale 1995,comedy drama romance,single mother based on novel or book clv reven...,114885,31357.0
4,5,father of the bride part ii 1995,comedy,clv daughter contraception steve martin gyneco...,113041,11862.0
...,...,...,...,...,...,...
53447,288765,bird box barcelona 2023,horror thriller,apocalypse bad science postapocalyptic plot ho...,14400246,805320.0
53448,288779,don camillo monsignor 1961,comedy,don camillo series,54814,11580.0
53449,288849,colaholic 2018,no genres listed,short film addiction animation,8632066,628759.0
53450,288937,blue thermal 2022,animation comedy drama romance,anime,15304756,865767.0


In [7]:
# Behåll

tfidf_movies = filtered_movies.copy()
tfidf_movies["tfidf"] = filtered_movies["genres"] + " " + filtered_movies["tag"]
tfidf_movies

,movieId,title,genres,tag,imdbId,tmdbId,tfidf
0,1,toy story 1995,adventure animation children comedy fantasy,birthday party owned very good battery microph...,114709,862.0,adventure animation children comedy fantasy bi...
1,2,jumanji 1995,adventure children fantasy,time alternate timeline handcuffs jungle man p...,113497,8844.0,adventure children fantasy time alternate time...
2,3,grumpier old men 1995,comedy romance,moldy funny comedy old people that is actually...,113228,15602.0,comedy romance moldy funny comedy old people t...
3,4,waiting to exhale 1995,comedy drama romance,chick flick characters girl movie single mothe...,114885,31357.0,comedy drama romance chick flick characters gi...
4,5,father of the bride part ii 1995,comedy,fantasy steve martin comedy confidence regret ...,113041,11862.0,comedy fantasy steve martin comedy confidence ...
...,...,...,...,...,...,...,...
53447,288765,bird box barcelona 2023,horror thriller,tw suicide bad science postapocalyptic surviva...,14400246,805320.0,horror thriller tw suicide bad science postapo...
53448,288779,don camillo monsignor 1961,comedy,don camillo series,54814,11580.0,comedy don camillo series
53449,288849,colaholic 2018,no genres listed,short film addiction animation,8632066,628759.0,no genres listed short film addiction animation
53450,288937,blue thermal 2022,animation comedy drama romance,anime,15304756,865767.0,animation comedy drama romance anime


In [8]:
# Behåll

from sklearn.feature_extraction.text import TfidfVectorizer

text_data1 = filtered_movies["genres"].tolist()
vectorizer1 = TfidfVectorizer()
tfidf_matrix1 = vectorizer1.fit_transform(text_data1)

text_data2 = filtered_movies["tag"].tolist()
vectorizer2 = TfidfVectorizer()
tfidf_matrix2 = vectorizer1.fit_transform(text_data2)

In [9]:
# Behåll

from scipy.sparse import hstack
combined_matrix = hstack((tfidf_matrix1, tfidf_matrix2))

In [10]:
# Behåll

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = "cosine", algorithm = "auto")
model_knn.fit(combined_matrix)

NearestNeighbors(metric='cosine')

In [11]:
def get_recommendations(movie_id, combined_matrix, model_knn):
    # Get the row corresponding to the movie of interest
    movie = combined_matrix[movie_id, :]
    
    # Get the indices and distances of the nearest neighbors
    distances, indices = model_knn.kneighbors(movie.reshape(1, -1))
    
    # Return the movie titles corresponding to the nearest neighbors
    return filtered_movies["title"].iloc[indices[0]]

movie_name = "toy story 1995"
movie_id = filtered_movies[filtered_movies["title"] == movie_name].index[0]


print("Recommendations for movie:", filtered_movies["title"].iloc[movie_id])
list = (get_recommendations(movie_id, combined_matrix, model_knn))
list

Recommendations for movie: toy story 1995


0                                toy story 1995
2874                           toy story 2 1999
46683                          toy story 4 2019
21203            toy story toons small fry 2011
21202    toy story toons hawaiian vacation 2011
Name: title, dtype: object

In [96]:
""" Funkar bäst hittils -= TF-IDF =- """
# Behåll

def get_recommendations(movie_id, combined_matrix, model_knn, top_n = 5):
    # Get the row corresponding to the movie of interest
    movie = combined_matrix[movie_id, :]
    
    # Get the indices and distances of the nearest neighbors
    distances, indices = model_knn.kneighbors(movie.reshape(1, -1), n_neighbors= top_n+1)
    
    # Return the movie titles corresponding to the nearest neighbors
    return filtered_movies[["movieId","title", "genres", "tmdbId"]].iloc[indices[0][1:top_n+1]]

movie_name = "heat 1995"
movie_id = filtered_movies[filtered_movies["title"] == movie_name].index[0]


print("Recommendations for movie:", filtered_movies["title"].iloc[movie_id])
list = get_recommendations(movie_id, combined_matrix, model_knn, top_n=50)
list

Recommendations for movie: heat 1995


,movieId,title,genres,tmdbId
6878,7298,code the mentale la 2002,action crime thriller,54045.0
46531,200756,living dying 2007,action crime thriller,17954.0
2066,2278,ronin 1998,action crime thriller,8195.0
28865,140679,bloody friday 1972,action crime drama thriller,28358.0
27409,135823,dhoom 3 2013,action crime thriller,44977.0
48696,217903,the last days of american crime 2020,action crime thriller,339095.0
16450,90751,son of no one the 2011,action crime thriller,74536.0
52312,272211,ambulance 2022,action crime thriller,763285.0
15858,87785,takers 2010,action crime thriller,22907.0
38954,171763,baby driver 2017,action crime thriller,339403.0


In [97]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [98]:
list
top50_n_ratings = pd.merge(list, ratings, on="movieId", how="inner")
top50_n_ratings

,movieId,title,genres,tmdbId,userId,rating,timestamp
0,7298,code the mentale la 2002,action crime thriller,54045.0,21341,3.5,1171948851
1,7298,code the mentale la 2002,action crime thriller,54045.0,42565,3.0,1077285821
2,7298,code the mentale la 2002,action crime thriller,54045.0,80664,3.0,1546204658
3,7298,code the mentale la 2002,action crime thriller,54045.0,110027,2.0,1586816442
4,7298,code the mentale la 2002,action crime thriller,54045.0,122731,2.5,1138030362
...,...,...,...,...,...,...,...
422335,95572,cruel gun story kenju zankoku monogatari 1964,action crime thriller,71722.0,286544,1.0,1625921322
422336,95572,cruel gun story kenju zankoku monogatari 1964,action crime thriller,71722.0,310006,4.0,1488817663
422337,95572,cruel gun story kenju zankoku monogatari 1964,action crime thriller,71722.0,313291,2.5,1620423457
422338,95572,cruel gun story kenju zankoku monogatari 1964,action crime thriller,71722.0,324508,3.0,1688703131


In [99]:
top_mean = top50_n_ratings.groupby(["movieId","title", "tmdbId"])["rating"].agg(["count", "mean"]).reset_index()
sorted_mean = top_mean.nlargest(5, "mean")
sorted_mean

,movieId,title,tmdbId,count,mean
36,158111,revenge 1978,224079.0,1,5.000000
2,293,leon the professional aka the professional leo...,101.0,43539,4.097545
4,1036,die hard 1988,562.0,47472,3.942671
16,54286,bourne ultimatum the 2007,2503.0,28138,3.908913
11,6874,kill bill vol 1 2003,24.0,46973,3.856407


In [ ]:
""" ============================================================================================="""

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

combined_matrix

vectorizer = TfidfVectorizer()
model = vectorizer.fit_transform(tfidf_movies["tfidf"])

In [ ]:
import numpy as np

# def search(title):
title = "toy story 1995"
query_vector = vectorizer.transform([title])
similarity = cosine_similarity(query_vector, model).flatten()
indices = np.argpartition(similarity, -50)[-50:]
result = filtered_movies.iloc[indices][["title", "genres"]].values

result

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value= "toy story 1995",
    description = "Movie:",
    disabled = False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 3:
            display(search(title))

movie_input.observe(on_type, names="value")

display(movie_input, movie_list)

Text(value='mortal kombat 1995', description='Movie:')